<a href="https://colab.research.google.com/github/jaya-shankar/education-impact/blob/master/life_expectancy_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model Performace History

1.   Cosidering life expectancy as input
      
      1. *Training using max no of countries:*\
          **MSE**: 1.421960711479187
          
      2. *Training using min no of countries:*\
          **MSE**: 1.6800711154937744

      3. *Training using only common countries:*\
          **MSE**: 1.5712116956710815

2.   Not cosidering life expectancy as input
      1. *Trained using max no of countries:*\
          **MSE**: 1.9395439624786377

      2. *Trained using min no of countries*:\
          **MSE**: 2.161480665206909

      3. *Trained using only common countries*:\
          **MSE**: 2.10016131401062



In [171]:
!rm -rf education-impact
!rm education-impact

rm: cannot remove 'education-impact': No such file or directory


In [172]:
!git clone https://github.com/jaya-shankar/education-impact.git


Cloning into 'education-impact'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 318 (delta 157), reused 157 (delta 46), pack-reused 0
Receiving objects: 100% (318/318), 2.11 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [173]:
# !pip install tensorflow_decision_forests
# !pip install wurlitzer
# !pip install seaborn

In [174]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from wurlitzer import sys_pipes

In [226]:
#@title Default title text
root = "education-impact/" 
datasets_path = {
                    "infant_mortality"              :  root+ "datasets/Infant_Mortality_Rate.csv",
                    "child_mortality"               :  root+ "datasets/child_mortality_0_5_year_olds_dying_per_1000_born.csv",
                    "children_per_woman"            :  root+ "datasets/children_per_woman_total_fertility.csv",
                    "co2_emissions"                 :  root+ "datasets/co2_emissions_tonnes_per_person.csv",
                    "population"                    :  root+ "datasets/converted_pop.csv",
                    "food_supply"                   :  root+ "datasets/food_supply_kilocalories_per_person_and_day.csv",
                    "gdp_growth"                    :  root+ "datasets/gdp_per_capita_yearly_growth.csv",
                    "Avg_daily_income_ppp"          :  root+ "datasets/mincpcap_cppp.csv",
                    "gdppercapita_us_infla_adjust"  :  root+ "datasets/gdppercapita_us_inflation_adjusted.csv",
                    "gini_index"                    :  root+ "datasets/gini.csv",
                    "life_expectancy"               :  root+ "datasets/life_expectancy_years.csv",
                    "malnutrition"                  :  root+ "datasets/malnutrition_weight_for_age_percent_of_children_under_5.csv",
                    "poverty_index"                 :  root+ "datasets/mincpcap_cppp.csv",
                    "maternal_mortality"            :  root+ "datasets/mmr_who.csv",
                    "people_in_poverty"             :  root+ "datasets/number_of_people_in_poverty.csv",
                    "primary_completion"            :  root+ "datasets/primary_school_completion_percent_of_girls.csv",
                    "ratio_b/g_in_primary"          :  root+ "datasets/ratio_of_girls_to_boys_in_primary_and_secondary_education_perc.csv",
                    "wcde-25--34"                   :  root+ "datasets/wcde-25--34.csv",
                    "wcde-Incomplete_Primary"       :  root+ "datasets/wcde-Incomplete Primary.csv",
                    "wcde-Lower_Secondary"          :  root+ "datasets/wcde-Lower Secondary.csv",
                    "wcde-Post_Secondary"           :  root+ "datasets/wcde-Post Secondary.csv",
                    "wcde_female-Incomplete_Primary":  root+ "datasets/wcde_female-Incomplete Primary.csv",
                    "wcde_female-Lower_Secondary"   :  root+ "datasets/wcde_female-Lower Secondary.csv",
                    "wcde_female-Post_Secondary"    :  root+ "datasets/wcde_female-Post Secondary.csv",
                    "wcde-In_Primary_OL"            :  root+ "datasets/wcde-In_Primary_OL.csv",
                    "wcde-Primary_OL"               :  root+ "datasets/wcde-Primary_OL.csv",
                    "wcde-Lower_Secondary_OL"       :  root+ "datasets/wcde-Lower_Secondary_OL.csv",
                    "female_wcde-In_Primary_OL"     :  root+ "datasets/wcde-female_In_Primary_OL.csv",
                    "female_wcde-Primary_OL"        :  root+ "datasets/wcde-female_Primary_OL.csv",
                    "female_wcde-Lower_Secondary_OL":  root+ "datasets/wcde-female_Lower_Secondary_OL.csv"
                }

In [301]:
datasets = [
            # "infant_mortality",
            # "life_expectancy",
            # "child_mortality",
            "children_per_woman",
            "co2_emissions",
            "gini_index",
            "gdppercapita_us_infla_adjust",
            "wcde-In_Primary_OL",
            "wcde-Primary_OL",
            # "wcde-Lower_Secondary_OL",
            "population",
            "female_wcde-In_Primary_OL",
            "female_wcde-Primary_OL" ,
            # "female_wcde-Lower_Secondary_OL"
            ]

In [302]:
PREDICT_FUTURE  = 10
OUTPUTS         = ['life_expectancy']
                   

In [303]:
# to find out how many countries each dataset has
countries_count      = None
least_dataset_path   = None
for dataset in datasets:
  df = pd.read_csv(datasets_path[dataset])
  count = len(set(df.Country.unique()))
  if not countries_count:
    countries_count = count
    least_dataset_path   = datasets_path[dataset]
  elif countries_count > count:
    countries_count = count
    least_dataset_path   = datasets_path[dataset]
  print(f"{'Factor: ' + dataset:<30} count: {count}")
print(f"{'To use: ' + least_dataset_path:<30} count: {countries_count}")

Factor: children_per_woman     count: 202
Factor: co2_emissions          count: 194
Factor: gini_index             count: 195
Factor: gdppercapita_us_infla_adjust count: 207
Factor: wcde-In_Primary_OL     count: 202
Factor: wcde-Primary_OL        count: 202
Factor: population             count: 197
Factor: female_wcde-In_Primary_OL count: 202
Factor: female_wcde-Primary_OL count: 202
To use: education-impact/datasets/co2_emissions_tonnes_per_person.csv count: 194


In [304]:
common_countries = set()
for dataset in datasets:
  countries_list = list(pd.read_csv(datasets_path[dataset]).Country)
  countries_list = set(map(lambda x: x.lower(), countries_list))
  if common_countries == set():
    common_countries = countries_list
  else:
    common_countries = common_countries.intersection(countries_list)
len(common_countries)

158

In [305]:
# creating a list of all countries & years
countries = list(common_countries)
years     = [y for y in range(1960,2015-PREDICT_FUTURE+1)]

In [306]:
len(countries)

158

In [307]:
keys=[]
for y in years:
  for c in countries:
    keys.append((c,str(y)))

In [308]:
big_dic = {k : [] for k in keys}
for dataset in datasets:
  df = pd.read_csv(datasets_path[dataset])
  df["Country"] = df["Country"].str.lower()
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][k[1]])
    except:
      big_dic[k].append(np.NaN)
 

In [309]:
for output_path in OUTPUTS:
  df = pd.read_csv(datasets_path[output_path])
  df["Country"] = df["Country"].str.lower()
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][str(int(k[1])+PREDICT_FUTURE)])
    except:
      big_dic[k].append(np.NaN)

In [310]:
columns = [k for k in datasets ]
output_columns = ["o_"+o for o in OUTPUTS]
columns.extend(output_columns)

In [311]:
input_df = pd.DataFrame.from_dict(big_dic,orient='index', columns = columns)
output_df = input_df[["o_"+o for o in OUTPUTS]]
input_df.drop(labels=["o_"+o for o in OUTPUTS], axis = 1, inplace=True)

In [312]:
X_train, X_test, y_train, y_test = train_test_split(input_df, output_df, test_size=0.30, random_state=43)

In [313]:
X_train.isna().sum()

children_per_woman                 0
co2_emissions                     97
gini_index                         0
gdppercapita_us_infla_adjust    1299
wcde-In_Primary_OL                 0
wcde-Primary_OL                    0
population                         0
female_wcde-In_Primary_OL          0
female_wcde-Primary_OL             0
dtype: int64

In [314]:
y_train.isna().sum()

o_life_expectancy    0
dtype: int64

In [315]:
X_train

,children_per_woman,co2_emissions,gini_index,gdppercapita_us_infla_adjust,wcde-In_Primary_OL,wcde-Primary_OL,population,female_wcde-In_Primary_OL,female_wcde-Primary_OL
"(ethiopia, 1966)",6.87,0.0322,52.3,NaN,98.14,99.40,25600000,98.14,99.40
"(malawi, 1963)",7.02,0.0882,48.3,178.0,92.18,95.98,3930000,92.18,95.98
"(montenegro, 1987)",2.12,3.0400,30.2,NaN,1.10,3.52,617000,1.10,3.52
"(pakistan, 2005)",4.07,0.8410,32.1,1180.0,61.90,70.80,160000000,61.90,70.80
"(papua new guinea, 1986)",5.11,0.4930,54.7,1740.0,0.00,0.00,4190000,0.00,0.00
...,...,...,...,...,...,...,...,...,...
"(paraguay, 1999)",3.68,0.8400,54.7,4080.0,19.22,53.58,5220000,19.22,53.58
"(belarus, 1974)",2.19,9.7300,22.4,NaN,0.24,2.74,9190000,0.24,2.74
"(sao tome and principe, 1974)",6.53,0.2260,32.1,NaN,90.88,96.86,81100,90.88,96.86
"(nicaragua, 1981)",6.02,0.6360,48.6,1890.0,55.08,74.24,3360000,55.08,74.24


In [316]:
# sns.pairplot(X_train[["infant_mortality","life_expectancy"]], diag_kind="kde")

In [317]:
# sns.pairplot(X_train[["infant_mortality","Avg_daily_income_ppp"]], diag_kind="kde")

In [318]:
# sns.pairplot(X_train[["infant_mortality","wcde-Incomplete_Primary"]], diag_kind="kde")

In [319]:
# sns.pairplot(X_train[["infant_mortality","population"]], diag_kind="kde")

In [320]:
y_train

,o_life_expectancy
"(ethiopia, 1966)",44.1
"(malawi, 1963)",44.2
"(montenegro, 1987)",73.8
"(pakistan, 2005)",64.4
"(papua new guinea, 1986)",63.4
...,...
"(paraguay, 1999)",75.8
"(belarus, 1974)",70.5
"(sao tome and principe, 1974)",61.4
"(nicaragua, 1981)",72.1


In [321]:
combine_dfs("o_"+OUTPUTS[0],X_test,y_test)

,children_per_woman,co2_emissions,gini_index,gdppercapita_us_infla_adjust,wcde-In_Primary_OL,wcde-Primary_OL,population,female_wcde-In_Primary_OL,female_wcde-Primary_OL,o_life_expectancy
"(iraq, 1963)",6.69,1.1900,43.4,NaN,86.40,92.66,7890000,86.40,92.66,64.7
"(argentina, 1974)",3.26,3.7500,35.5,10100.0,19.22,57.00,25500000,19.22,57.00,70.8
"(montenegro, 1992)",2.05,2.4000,30.2,NaN,1.06,2.68,617000,1.06,2.68,73.9
"(italy, 1975)",2.12,6.1800,37.8,17800.0,2.70,24.00,55300000,2.70,24.00,75.8
"(switzerland, 1960)",2.52,3.6900,36.8,NaN,4.40,20.20,5280000,4.40,20.20,73.3
...,...,...,...,...,...,...,...,...,...,...
"(ireland, 1977)",3.34,7.2700,35.9,15100.0,0.34,23.28,3240000,0.34,23.28,74.3
"(nepal, 1993)",4.93,0.0718,33.8,464.0,70.52,80.30,20500000,70.52,80.30,67.0
"(new zealand, 2004)",2.02,8.8200,33.2,34300.0,1.24,4.62,4080000,1.24,4.62,81.5
"(central african republic, 1970)",5.95,0.1150,70.7,655.0,99.10,99.80,1810000,99.10,99.80,49.2


In [322]:
def combine_dfs(label,X,y):
  frames      = [X,y[label]]
  le_model_df = pd.concat(frames,axis=1)
  le_model_df.dropna(subset=[label],inplace=True)
  return le_model_df

In [323]:
for output in OUTPUTS:
  # converting pandas to tensorFlow dataset
  train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(combine_dfs("o_"+output,X_train,y_train), label="o_"+output, task=tfdf.keras.Task.REGRESSION)
  model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
  
  model.fit(x=train_ds)
  # Convert it to a TensorFlow dataset
  test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(combine_dfs("o_"+output,X_train,y_train), label="o_"+output, task=tfdf.keras.Task.REGRESSION)

  # Evaluate the model
  model.compile(metrics=["mse"])
  # Evaluate the model on the test dataset.
  evaluation = model.evaluate(test_ds, return_dict=True)
  print(output.upper())
  print(evaluation)
  print()
  print(f"MSE: {evaluation['mse']}")
  print(f"RMSE: {math.sqrt(evaluation['mse'])}")
  print()

80/80 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - mse: 2.2808
LIFE_EXPECTANCY
{'loss': 0.0, 'mse': 2.2807931900024414}

MSE: 2.2807931900024414
RMSE: 1.5102295156705292



In [324]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)

In [325]:
# %set_cell_height 300

model.summary()

Model: "random_forest_model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (9):
	children_per_woman
	co2_emissions
	female_wcde-In_Primary_OL
	female_wcde-Primary_OL
	gdppercapita_us_infla_adjust
	gini_index
	population
	wcde-In_Primary_OL
	wcde-Primary_OL

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.                      "__LABEL" 11.454764 ################
    2.           "wcde-In_Primary_OL"  8.076121 #########
    3.    "female_wcde-In_Primary_OL"  7.876849 #########
    4.                   "population"  7.036748 ########
    5.                   "gini_index"  5.905103 ######
    6.                "co2_emissions"  5.685592 #####
    7.       "female_wcde-Primary_OL"  5.541573 #####
    8

# CNN Algorithm

from the above output
- **malnutrition & people in povery** have least no of countries
- **infant mortality & gdp per captia** have highest no of countries

*Doubt:* Does having more data for one factor will make the decision tree bias?


###Steps
1. create a csv file such that each row contains all values of particular year & country present
2. the output for each row is year + 40 years corresponding value 
    1. **outputs** - life expectany, education level, gdp




From above output
- if we dont drop any rows our table size = 4256 entries
- if we drop rows containing any if all of outputs missing then our table size = 3039 entries
- if we drop rows containing any one of output missing then our table size = 1745 entries

so, I think its is better to go with second choice and build different models, but not sure it will not effect performance of the model


now we have the dataframe containing both inputs and ouputs,our next step is
1. split the data into train & test data
  1. try to split data based on continents to reduce bias
2. build DF model using tensorflow
3. check the accuracy of the model